In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [2]:
9*4

36

In [8]:
class Person:
    '''This class can be used to represent a person'''
    def__init__(self,name=None):
        self._name=name
    def set_name(self,name):
          assert(isinstance(name,str))
          if isinstance(name,str):
            sekf.__name=name
          else:
            print('not valid')
    def get_name(self):
        return self._name
    def__greet_once(self):
        print("hello,I'm{}".format(self.__name))
    def greet(self):
        self._greet_once()
        self._greet_once()

zs=Persom('zhang san')
greet()

SyntaxError: invalid syntax (<ipython-input-8-a6b8efb5347b>, line 3)

In [9]:
1+2

3

In [10]:
print('hello'.count('e'))
print([1,2,3,'e','e',4].count('e'))

1
2


In [11]:
my_list=['hello',[1,2,3,'e','e',4]]
for item in my_list:
    print(item.count('e'))

1
2


In [12]:
class Cat:
    def talk(self):
        print('meow...')
class Dog:
    def talk(self):
        print('wang,wang...')

pets=[Cat(),Dog(),Cat()]
for item in pets:
        item.talk()
        

meow...
wang,wang...
meow...


In [13]:
def add(a,b):
    return a+b
a=1
b=2
print(add(a,b))
x,y='a','b'
print(add(x,y))

3
ab


In [16]:
#inheritence
class Student(Person):
    def set_score(self,score):
        self.__score=score
    def get_score(self):
        return self._score
    def show_score(self):
        print('My score is:{}'.format(self.__score))
    def greet(self):
        Person.greet(self)
        self.show_score()
zs=Student()
zs.set_name("Zhang San")
zs.get_score(100)
zs.greet()
#zs.show_score()

NameError: name 'Person' is not defined